<a href="https://colab.research.google.com/github/abaskon/Particle_Filter_Localisation/blob/main/Particle_Filter_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Particle Filter

**Frank Dellaert, March 7th 2021**

This a quick example of a particle filter in 2D. Because the example is robot localization, this is also an example of ["Monte Carlo Localization" or MCL](https://www.ri.cmu.edu/publications/monte-carlo-localization-for-mobile-robots/).

In [1]:
import numpy as np
import plotly.express as px

## Prior knowledge as samples

Let's assume we have a robot living in a 2D world, but we only have a vague idea of where the robot starts out. In particular, let's assume we only know that we are somewhere near the origin, as represented by a Gaussian with a pretty wide standard deviation:

In [2]:
N=300
mean = np.array([0,0])
covariance = np.diag([9,9])
samples = np.random.multivariate_normal(mean, covariance, size=N)

In [3]:
fig = px.scatter(x=samples[:,0], y=samples[:,1])
fig.update_layout(width = 500, height = 500, title = "Samples from prior")
fig.update_yaxes(range=[-10,10], scaleanchor = "x",scaleratio = 1) # axis equal
fig.show()

## Update with information from a measurement

We can upgrade this sample representation of the *prior* to a weighted sample of the *posterior*, using [importance sampling](https://en.wikipedia.org/wiki/Importance_sampling) to implement [Bayes law](https://en.wikipedia.org/wiki/Bayes%27_theorem).

Let us assume we have a sensor that *only* give us information about or x position, corrupted by Gaussian noise with standard deviation $\sigma$. Then *given* a measurement $m$, the corresponding likelihood function $l(x, y; m)$ of the robot position $(x,y)$ is
$$l(x,y;m)=\exp-0.5 \frac{(x-m)^2}{\sigma^2}$$

To implement Bayes law with importance sampling, the importance weight we use should be the ratio of *target* density, which is the posterior $p(x,y|m)\propto l(x,y;m) p(x,y)$, and the *proposal* density for which we use the prior $p(x,y)$, so **the weight turns out to be just the likelihood**:
$$w(x,y;m) = \frac{p(x,y|m)}{p(x,y)} = \frac{l(x,y;m) p(x,y)}{p(x,y)} = l(x,y;m)$$

Let's implement this with $m=3$ and $\sigma=1$:

In [4]:
m=3 # measurement
sigma=1 # standard deviation
variance = sigma**2
def likelihood(location): return np.exp(-0.5*(location[0]-m)**2/variance)

In [5]:
weights = np.apply_along_axis(likelihood, 1, samples)

In [6]:
fig = px.scatter(x=samples[:,0], y=samples[:,1], size=weights)
fig.update_layout(width = 500, height = 500, title = "Weighted samples from posterior")
fig.update_yaxes(range=[-10,10], scaleanchor = "x",scaleratio = 1) # axis equal
fig.show()

As you can see above, the posterior is still very uncertain about where the robot is.

## Predict with a motion model

The second phase in a particle filter is predicting a new set of (unweighted) samples using a model for how the robot moves. This then will act as the prior in the next time step.

In this example, let us assume the robot moves in a circle around the origin, in a clockwise direction. In other words, the velocity associated with a state $(x,y)$ will be $v=(-y, x)*V/\sqrt{x^2+y^2}$, where V is the forward velocity. We can simulate each particle's motion forward with a few simple Euler steps:

In [7]:
# Simulate all samples forward for one second, using 10 Euler steps:
V=5
predictions = np.copy(samples)
for i in range(10):
  x = predictions[:,0]
  y = predictions[:,1]
  norm = np.sqrt(x**2 + y**2)
  predictions[:,0] -= 0.1*y*V/norm
  predictions[:,1] += 0.1*x*V/norm

The rotated samples are shown below:

In [8]:
fig = px.scatter(x=predictions[:,0], y=predictions[:,1], size=weights)
fig.update_layout(width = 500, height = 500, title = "Weighted samples from posterior")
fig.update_yaxes(range=[-10,10], scaleanchor = "x",scaleratio = 1) # axis equal
fig.show()

## Resample to get unweighted samples again
The above moved all the weighted samples to weighted predictions. However, to apply the measurement update, we need *unweighted* samples. To accomplish this, we use `np.random.choice` again:

In [9]:
sample_indices = np.random.choice(len(samples),p=weights/np.sum(weights),size=N)
samples = predictions[sample_indices]

In [10]:
fig = px.scatter(x=samples[:,0], y=samples[:,1])
fig.update_layout(width = 500, height = 500, title = "Reweighted samples")
fig.update_yaxes(range=[-10,10], scaleanchor = "x",scaleratio = 1) # axis equal
fig.show()

## Recurse: update with a new measurement

In [11]:
m=5 # measurement
def likelihood(location): return np.exp(-0.5*(location[0]-m)**2/variance)

In [12]:
weights = np.apply_along_axis(likelihood, 1, samples)

In [13]:
fig = px.scatter(x=samples[:,0], y=samples[:,1], size=weights)
fig.update_layout(width = 500, height = 500, title = "Weighted samples from posterior")
fig.update_yaxes(range=[-10,10], scaleanchor = "x",scaleratio = 1) # axis equal
fig.show()

As you can see, we are now pretty well localized, after only 2 measurement updates. The key is that the measurement and motion models worked together to provide a consistent posterior approximation.

We can now repeat the cycle by predicting again, updating again, etc...